In [39]:
# padronizar a sequência inserida pelo usuário (formato FASTA ou sequência livre de nucleotídeos).
# insere um * no início da sequência para construçã da matriz para o alinhamento.

def formata_sequencia(sequencia): 
    sequencia = sequencia.upper()
        
    if sequencia[0] == ">":
        sequencia = sequencia.splitlines()
        sequencia = sequencia[1:]
        sequencia = "".join(sequencia).strip()
        sequencia = "*" + sequencia
        
    else:
        sequencia = sequencia.splitlines()
        sequencia = "".join(sequencia).strip()
        sequencia = "*" + sequencia 
        
    return sequencia


In [40]:
'''
Retorna o maior valor do L
'''

def valor_maximo(c1, c2, lado, cima, diagonal):

    # retorna o valor da diagonal superior esquerda + 1 
    if (c1 == c2) and (diagonal > lado) and (diagonal > cima):
        return diagonal

    # retorna o valor do lado esquerdo
    elif (lado > cima) and (lado > diagonal):
        return lado
  
    # retorna o valor de cima
    else:
        return cima


'''
Retorna o ponteiro do maior valor do L
'''

def acha_caminho(c1, c2, lado, cima, diagonal):
    # indica que o valor veio da diagonal esquerda superior
    if (c1 == c2) and (diagonal > lado) and (diagonal > cima):
        return "\\"

    # indica que o valor veio do lado esquerdo
    elif (lado > cima) and (lado > diagonal):
        return "-"
        
    # indica que o valor veio de cima
    else:
        return "|"


In [41]:
'''
Montar as matrizes:
- pontuacao (recebe os valores dos matches)
- caminho (recebe o caminho percorrido na matriz de acordo com o maior valor)

- seq1 fica na vertical, seq2 na horizontal
'''

def lcs(seq1, seq2):
    pontuacao = []
    caminho = []

    # preencher as linhas da matriz
    for i in range(0, len(seq1)):
        pontuacao.append([0] * len(seq2))
        caminho.append([""] * len(seq2))

    # preencher a primeira coluna com "|" e a primeira linha com "-" na matriz caminho
    for i in range(0, len(seq1)):
        pontuacao[i][0] = -3 * i
        caminho[i][0] = "|"
    for j in range(0, len(seq2)):
        pontuacao[0][j] = -3 * j
        caminho[0][j] = "-"

    # percorre as linhas
    for i in range(1, len(seq1)):

        # percorre as colunas
        for j in range(1, len(seq2)):
            c1 = seq2[j]
            c2 = seq1[i]
            lado = pontuacao[i][j-1] - 3
            cima = pontuacao[i-1][j] - 3
            diagonal = pontuacao[i-1][j-1] + 4

        # preenche a matriz com o maior valor do L (lado, cima, diagonal)
            pontuacao[i][j] = valor_maximo(c1, c2, lado, cima, diagonal)

        # preenche a matriz com a indicação de onde veio o maior valor do L (lado, cima, diagonal)
            caminho[i][j] = acha_caminho(c1, c2, lado, cima, diagonal)
        
    return pontuacao, caminho

In [42]:
'''
Imprimir a matriz
'''

def imprime_matriz(seq1, seq2, pontuacao, caminho):
    print("\t", end="")

    # imprime os caracteres da sequencia 2 na linha superior (horizontal) 
    for j in range(0, len(seq2)):
        print(seq2[j], end="\t")

    print()

    # imprime os caracteres da sequencia 1 na primeira coluna (vertical) e vai preenchendo as linhas com os valores
    for i in range(0, len(seq1)):
        print(seq1[i], end="\t")
        for j in range(0, len(seq2)):
            print(pontuacao[i][j], caminho[i][j], end="\t", sep="")

        print()
    print()


In [43]:
'''
Alinha as sequências
'''

def gera_alinhamento(seq1, seq2, pontuacao, caminho):
    ali_seq1 = ""
    ali_seq2 = ""

    # linhas
    i = len(seq1)-1
    # colunas
    j = len(seq2)-1

    while (i != 0) or (j != 0):
        # \\ indica que houve um match (imprime o mesmo caracter nas duas seqências)
        if caminho[i][j] == "\\":
            ali_seq1 = seq1[i] + ali_seq1
            ali_seq2 = seq2[j] + ali_seq2
            i -= 1
            j -= 1

        # indical uma deleção na seq1 (imprime o caracter da seq2)
        elif caminho[i][j] == "-":
            ali_seq1 = "-" + ali_seq1
            ali_seq2 = seq2[j] + ali_seq2
            j -= 1
            
        # indical uma deleção na seq2 (imprime o caracter da seq1)
        else:
            ali_seq1 = seq1[i] + ali_seq1
            ali_seq2 = "-" + ali_seq2
            i -= 1

    # imprime a pontuação (total de matches)
    print("Matches =", pontuacao[-1][-1])
    # imprime o alinhamento
    print(ali_seq1)
    print(ali_seq2)
      

In [44]:
seq1 = "AATCGATCGAAT"
seq2 = "AACTCGACGCT"

seq1 = formata_sequencia(seq1)
seq2 = formata_sequencia(seq2)


In [45]:
matriz_pontuacao, matriz_caminho = lcs(seq1, seq2)
imprime_matriz(seq1, seq2, matriz_pontuacao, matriz_caminho)
gera_alinhamento(seq1, seq2, matriz_pontuacao, matriz_caminho)

	*	A	A	C	T	C	G	A	C	G	C	T	
*	0-	-3-	-6-	-9-	-12-	-15-	-18-	-21-	-24-	-27-	-30-	-33-	
A	-3|	4\	-9|	-12|	-15|	-18|	-21|	-14\	-27|	-30|	-33|	-36|	
A	-6|	1|	8\	5-	2-	-1-	-4-	-7-	-30|	-33|	-36|	-39|	
T	-9|	-2|	5|	2|	9\	-4|	-7|	-10|	-33|	-36|	-39|	-32\	
C	-12|	-5|	2|	9\	6|	13\	10-	7-	4-	1-	-2-	-5-	
G	-15|	-8|	-1|	6|	3|	10|	17\	4|	1|	8\	-5|	-8|	
A	-18|	-11|	-4|	3|	0|	7|	14|	21\	18-	15-	-8|	-11|	
T	-21|	-14|	-7|	0|	7\	4|	11|	18|	15|	12|	-11|	-4\	
C	-24|	-17|	-10|	-3|	4|	11\	8|	15|	22\	9|	16\	13-	
G	-27|	-20|	-13|	-6|	1|	8|	15\	12|	19|	26\	23-	10|	
A	-30|	-23|	-16|	-9|	-2|	5|	12|	19\	16|	23|	20|	7|	
A	-33|	-26|	-19|	-12|	-5|	2|	9|	16|	13|	20|	17|	4|	
T	-36|	-29|	-22|	-15|	-8|	-1|	6|	13|	10|	17|	14|	21\	

Matches = 21
AA-TCGATCG-AAT
AACTCGA-CGC--T
